In [164]:
import json
from landsatxplore.api import API
import getpass
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from hublib.ui import FileUpload
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import FileUpload, Checkbox, FloatSlider, Dropdown, Button, Output, VBox, DatePicker
import urllib.request
from io import BytesIO
from zipfile import ZipFile
from datetime import datetime
import os
from IPython.display import display, HTML

In [165]:
import hublib.ui as ui
from hublib.ui import HideCodeButton, RunAllButton

In [166]:
RunAllButton()

Button(description='Run All Cells', style=ButtonStyle())

In [167]:
RunAllButton(label="Run Cells", icon='fa-play-circle-o', style='danger', tooltip='Run All Cells', hide=True)

Button(button_style='danger', description='Run Cells', icon='play-circle-o', style=ButtonStyle(), tooltip='Run…

In [168]:
HideCodeButton()

Button(description='Hide Code Cells', style=ButtonStyle())

In [169]:
# called when all files finish uploading
def done_cb(w, name):
    # Do something with the files here...
    # We just print their names
    print("%s uploaded" % name)
    
    # reset clears and re-enables the widget for more uploads
    # You may want to do this somewhere else, depending on your GUI
    w.reset()

In [170]:
# username_input = widgets.Text(description='Enter your Earth Explorer username: ')
# password_input = widgets.Password(description='Enter your Earth Explorer password: ')
username = input('Enter your Earth Explorer username: ')
password = getpass.getpass('Enter your Earth Explorer password: ')
# bbox_file_input = widgets.FileUpload(description='Upload Bbox Shapefile')

Enter your Earth Explorer username: maggiechen259
Enter your Earth Explorer password: ········


In [171]:
# Process uploaded bbox shapefile
bbox_filename = input('Enter the name of the uploaded bounding box shapefile (with extension): ')
bbox_file_path = f'bbox/{bbox_filename}.shp'

Enter the name of the uploaded bounding box shapefile (with extension): test_bbox


In [172]:
# # this will allow you to select a single file to be uploaded
# f = FileUpload("Upload Area of Interest", 
#                "This is a description that appears when the mouse hovers over the name.",
#                dir='tmpdir',
#                cb=done_cb,
#                maxsize='20M')
# f
# File upload widget

file_upload = FileUpload(
    description="Upload Shapefile",
    accept=".shp",
    multiple=False
)
file_upload

FileUpload(value={}, accept='.shp', description='Upload Shapefile')

In [173]:
# output_dir = './box'
# local_file_path = os.path.join(output_dir, f'{ID}.shp')


In [174]:
def handle_file_upload(change):
    with output:
        # Clear previous output
        output.clear_output()

        # Check if any files are uploaded
        if file_upload.value:
            # Get the first uploaded file content
            uploaded_file_content = next(iter(file_upload.value.values()))['content']

            # Create a file-like object from the uploaded content
            file_like_object = BytesIO(uploaded_file_content)

            # Extract the zip file if it's a shapefile archive
            with ZipFile(file_like_object, 'r') as zip_ref:
                # Assuming the shapefile is in the root of the zip file
                shapefile_name = [name for name in zip_ref.namelist() if name.endswith('.shp')][0]

                # Read the shapefile using geopandas
                bbox = gpd.read_file(BytesIO(zip_ref.read(shapefile_name)))

            # Extract the bounds from the shapefile
            bounds = bbox.bounds.iloc[0].to_list()

            print("File uploaded successfully.")
            print("Bounds:", bounds)

In [175]:
# # Attach the handle_file_upload function to the upload button's on_click event
# upload_button.on_click(handle_file_upload)

In [176]:
message = "Please choose within two weeks"
display(HTML(f"<p style='color:red'>{message}</p>"))

In [177]:
start_date = DatePicker(
    description='Start Date',
    disabled = False,
    layout={'width': '300px'}
)
start_date

DatePicker(value=None, description='Start Date', layout=Layout(width='300px'))

In [178]:
end_date = DatePicker(
    description='End Date',
    disabled = False,
    layout={'width': '300px'}
)
end_date

DatePicker(value=None, description='End Date', layout=Layout(width='300px'))

In [179]:
# Create multiple checkboxes
check1 = ui.Checkbox('Landsat 5', desc='Landsat 5', value=False)
check2 = ui.Checkbox('Landsat 7', desc='Landsat 7', value=False)
check3 = ui.Checkbox('Landsat 8', desc='Landsat 8', value=False)
check4 = ui.Checkbox('Landsat 9', desc='Landsat 9', value=False)

In [180]:
landsat = widgets.VBox([check1, check2, check3,check4])

# Display the checkboxes in a horizontal layout
display(landsat)

In [181]:
# Cloud cover
cloud_cover = ui.Number(
        name='Max Cloud Cover',
        description="Cloud Cover",
        units='%',
        min=0,
        max=500,
        value=0
)

# cloud_cover = ui.Number(name='Cloud Cover',value=0, min=0, max=100, step=1, description='Cloud Cover (%)')
# cloud_cover = FloatSlider(value=0, min=0, max=100, step=1, description='Cloud Cover (%)')
cloud_cover

Number(children=(HTML(value='<div data-toggle="popover" title="Cloud Cover\nValues must be in units of percent…

In [182]:
# composite color widgets
composite_color = ui.Dropdown(
            name='Composite Color',
            description="Choose the composite color",
            value='True Color',
            options={'True Color', 'False Color'},
#             cb=plot
        )
composite_color

# composite_color = Dropdown(options=['True Color', 'False Color'], value='True Color', description='Composite Color')

Dropdown(children=(HTML(value='<div data-toggle="popover" title="Choose the composite color" data-container="b…

In [183]:
# Output widget to display the result
result_output = Output()

In [184]:
# Button to trigger the search and display results
search_button = Button(description='Display')

In [185]:
# Function to handle button click event
def on_search_button_click(b):
    # Retrieve the file path from the FileUpload widget
    uploaded_files = file_upload.value
    if uploaded_files:
        shapefile_path = list(uploaded_files.keys())[0]
        
        # Read the uploaded shapefile
        bbox = gpd.read_file(shapefile_path)
        bounds = bbox.bounds.iloc[0].to_list()
        
        # Retrieve other widget values
        landsat_options = [check1.value, check2.value, check3.value, check4.value]
        cloud_cover_value = cloud_cover.value
        composite_color_value = composite_color.value
        
        # Perform further processing based on the retrieved values
        with result_output:
            print(f"Shapefile Path: {shapefile_path}")
            print(f"Landsat Options: {landsat_options}")
            print(f"Cloud Cover: {cloud_cover_value}%")
            print(f"Composite Color: {composite_color_value}")

In [186]:
# Attach the function to the button click event
display_button.on_click(on_search_button_click)

In [187]:
#upload area of interest

# bbox = gpd.read_file(file_upload.value[0])
bbox = gpd.read_file(bbox_file_path)

bounds = bbox.bounds.iloc[0].to_list()

In [188]:
# Display the widgets
VBox([file_upload,
#       upload_button, 
#       output,
      start_date,
      end_date,
      VBox([check1, check2, check3, check4]),
      cloud_cover, 
      composite_color, 
      search_button,
      result_output])


In [191]:
def display_table():
    df_scenes = pd.DataFrame(scenes)
    df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date']]
    df_scenes.sort_values('acquisition_date', ascending=False, inplace=True)
    df_scenes['acquisition_date'] = pd.to_datetime(df_scenes['acquisition_date'])
    display(df_scenes)

In [192]:
#Landsat 5 TM Collection 2 Level 1	landsat_tm_c2_l1
# Landsat 5 TM Collection 2 Level 2	landsat_tm_c2_l2
# Landsat 7 ETM+ Collection 2 Level 1	landsat_etm_c2_l1
# Landsat 7 ETM+ Collection 2 Level 2	landsat_etm_c2_l2
# Landsat 8 Collection 2 Level 1	landsat_ot_c2_l1
# Landsat 8 Collection 2 Level 2	landsat_ot_c2_l2
# Landsat 9 Collection 2 Level 1	landsat_ot_c2_l1
# Landsat 9 Collection 2 Level 2	landsat_ot_c2_l2

# dic = {5:['landsat_tm_c2_l1','landsat_tm_c2_l2'],
#        7:['landsat_etm_c2_l1','landsat_etm_c2_l2'],
#        8:['landsat_ot_c2_l1','landsat_ot_c2_l2'],
#        9:['landsat_ot_c2_l1','landsat_ot_c2_l2']}

dic = {5:'landsat_tm_c2_l1',
       7:'landsat_etm_c2_l1',
       8:'landsat_ot_c2_l1',
       9:'landsat_ot_c2_l1'}
landsats = [5,7,8,9]


# #Initialize a new API instance

# Get the values of checkboxes within the VBox
checkbox_values = [child.value for child in landsat.children]

# print(checkbox_values)
for index, value in enumerate(checkbox_values):
    if value == True:
        landsat_id = landsats[index]
print(landsat_id)
print(dic[landsat_id])

# if landsat.value in dic:
#     landsat_dataset_id = dic[landsat.value]
    
api = API(username, password)

# Perform a request
response = api.request(endpoint="dataset-catalogs")
print(response)

# Search for Landsat TM scenes
print(start_date.value)
print(end_date.value)
scenes = api.search(
#     dataset='landsat_ot_c2_l1',
    dataset = str(dic[landsat_id]),
#     dataset=str(checkbox_values[0]),
    bbox = bounds,
    start_date = str(start_date.value),
    end_date = str(end_date.value),
    max_cloud_cover= str(cloud_cover.value),
#     composite_color = composite_color.value,
#     landsat = [checkbox.value for checkbox in landsat.children]
)
print(f"{len(scenes)} scenes found.")

# Process the result
for scene in scenes:
    print(scene['acquisition_date'].strftime('%Y-%m-%d'))
    # Write scene footprints to disk
    fname = f"{scene['landsat_product_id']}.geojson"
    with open(fname, "w") as f:
        json.dump(scene['spatial_coverage'].__geo_interface__, f)

# log out
api.logout()



8
landsat_ot_c2_l1
{'EE': 'EarthExplorer', 'GV': 'GloVis', 'HDDS': 'HDDS Explorer'}
2020-08-20
2020-09-20
7 scenes found.
2020-09-12
2020-09-12
2020-09-03
2020-08-27
2020-08-25
2020-08-20
2020-08-20


In [193]:
# Create a DataFrame from the scenes
df_scenes = pd.DataFrame(scenes)
df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date']]
df_scenes.sort_values('acquisition_date', ascending=False, inplace=True)
df_scenes['acquisition_date'] = pd.to_datetime(df_scenes['acquisition_date'])

dates = []
for scene in scenes:
    dates.append(scene['acquisition_date'].strftime('%Y-%m-%d'))
#     print(scene['acquisition_date'].strftime('%Y-%m-%d'))
print(dates)
df_scenes

['2020-09-12', '2020-09-12', '2020-09-03', '2020-08-27', '2020-08-25', '2020-08-20', '2020-08-20']


,display_id,wrs_path,wrs_row,satellite,cloud_cover,acquisition_date
0,LC08_L1TP_010010_20200912_20200919_02_T1,10,10,8,32,2020-09-12
1,LC08_L1TP_010011_20200912_20200919_02_T1,10,11,8,41,2020-09-12
2,LC08_L1TP_011010_20200903_20200918_02_T1,11,10,8,39,2020-09-03
3,LC08_L1TP_010010_20200827_20200906_02_T1,10,10,8,3,2020-08-27
4,LC08_L1TP_012010_20200825_20200905_02_T1,12,10,8,36,2020-08-25
5,LC08_L1TP_009010_20200820_20200905_02_T1,9,10,8,0,2020-08-20
6,LC08_L1TP_009011_20200820_20200905_02_T1,9,11,8,0,2020-08-20


In [194]:
choose_date = [ui.Checkbox(date, value=False, description=date) for date in dates]

display_dates = widgets.VBox(choose_date)


# # Display the checkboxes in a horizontal layout
# landsat = widgets.VBox([check1, check2, check3,check4])

display(display_dates)

In [196]:
checkbox_dates = [child.value for child in display_dates.children]
# print(checkbox_dates)
# print(dates)

for index, date in enumerate(checkbox_dates):
    if date == True:
        chosen_date=dates[index]
        chosen_date_index = index
        print(chosen_date)
        print(chosen_date_index)
    


2020-09-12
1


In [197]:
# Attach the function to the button click event
display_button.on_click(display_table)
# Attach the function to the button click event using a lambda function
# display_button.on_click(lambda _: display_table())


In [198]:
# exe_button = widgets.Button(
#     description = 'Run Analysis',
#     disabled = False,
#     button_style = 'success'
# )
# exe_button.on_click(execute)

In [59]:
from landsatxplore.earthexplorer import EarthExplorer
import os
import time
import requests


# Initialize the API
api = EarthExplorer(username, password)

# Select the first scene
# Assuming the first scene ID is stored in the variable first_scene_id
ID = df_scenes['display_id'].iloc[0]


# Get the Earth Explorer website URL for the scene
api_scene_url = f'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&scene_id={ID}&node=INVSVC'


# Open the scene URL in a web browser for manual download
# print(f'You can manually download the scene from the following URL:')
# print(api_scene_url)

# Define the local file path
output_dir = './data'
local_file_path = os.path.join(output_dir, f'{ID}.tar')



# Download the scene using requests
try:
    response = requests.get(api_scene_url)
    if response.status_code == 200:
        with open(local_file_path, 'wb') as f:
            f.write(response.content)
        print(f'{ID} successfully downloaded.')
    else:
        print(f'{ID} download failed. HTTP status code: {response.status_code}')
except Exception as e:
    print(f'{ID} download failed. Error: {str(e)}')

api.logout()


NameError: name 'df_scenes' is not defined

In [ ]:
import tarfile

ID = 'LC08_L2SP_010010_20221004_20221012_02_T1'
file_path = './data/{}.tar'.format(ID)

try:
    tar = tarfile.open(file_path)
    tar.extractall('./data/{}'.format(ID))
    tar.close()
    print('Extraction successful')
except Exception as e:
    print(f'Error: {e}')
    
#unzip twice 


In [ ]:
# import tarfile

# # Extract files from tar archive
# tar = tarfile.open('./data/{}.tar'.format(ID))
# tar.extractall('./data/{}'.format(ID))
# tar.close()